In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#load the trained model, scaler pickele file and label encoder pickle file
model=load_model('ann_model.h5')


#load the encoder and scaler
with open('scaler.pkl','rb') as f:
    scaler=pickle.load(f)

with open('lable_encoder_gender.pkl','rb') as f:
    lable_encoder_gender=pickle.load(f)

with open('onehot_encoder_geography.pkl','rb') as f:
    onehot_encoder_geography=pickle.load(f)

In [26]:
lable_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender' : 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
input_df = pd.DataFrame([input_data])
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
#transform the geography column
input_gender_encoder = lable_encoder_gender.transform(input_df['Gender'])
input_df['Gender'] = input_gender_encoder

In [6]:
#transform the geography column using onehot encoder
input_geo_encoder = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()
input_geo_encoder_df = pd.DataFrame(input_geo_encoder,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))
input_geo_encoder_df.head()

/Users/ganeshkapkoti/Udemy_GenAI/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [25]:
onehot_encoder_geography.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [8]:
input_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
#concat the onehot encoded geography columns to the input dataframe and drop the original geography column
input_df_final = pd.concat([input_df.reset_index(drop=True),input_geo_encoder_df],axis=1)
input_df_final.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
input_df_final = input_df_final.drop(['Geography'],axis=1)
input_df_final.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [27]:
scaler.feature_names_in_

array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
#scalling this input data
input_data_scaled = scaler.transform(input_df_final)
input_data_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [13]:
#prediction churn or not
pred = model.predict(input_data_scaled)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step


array([[0.03373521]], dtype=float32)

In [15]:
pred_probablity = pred[0][0] 
pred_probablity

0.03373521